<a href="https://colab.research.google.com/github/CedricChauvet/holbertonschool-machine_learning/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Lambda
from keras.layers import Dropout
from keras.layers import Input
from keras.applications import InceptionResNetV2
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import img_to_array, load_img
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# from keras.utils.np_utils import to_categorical


In [16]:

def prepa():
     data_labels = pd.read_csv('./dog-breed-identification/labels.csv')
     train_folder = './dog-breed-identification/train/'
     data_labels['image_path'] = data_labels.apply(lambda row: (train_folder + row["id"] + ".jpg" ),
                                                  axis=1)


     # load dataset
     train_data = np.array([img_to_array(load_img(img, target_size=(299, 299)))
                              for img in data_labels['image_path'].values.tolist()
                         ]).astype('float32')


     target_labels = data_labels['breed']
     # create train and test datasets
     x_train, x_test, y_train, y_test = train_test_split(train_data, target_labels,
                                                       test_size=0.3,
                                                       stratify=np.array(target_labels),
                                                       random_state=42)

     # create train and validation datasets
     x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                       test_size=0.15,
                                                       stratify=np.array(y_train),
                                                       random_state=42)

     print('Initial Dataset Size:', train_data.shape)
     print('Initial Train and Test Datasets Size:', x_train.shape, x_test.shape)
     print('Train and Validation Datasets Size:', x_train.shape, x_val.shape)
     print('Train, Test and Validation Datasets Size:', x_train.shape, x_test.shape, x_val.shape)

     # hot one encode
     y_train_ohe = pd.get_dummies(y_train.reset_index(drop=True)).to_numpy()
     y_val_ohe = pd.get_dummies(y_val.reset_index(drop=True)).to_numpy()
     y_test_ohe = pd.get_dummies(y_test.reset_index(drop=True)).to_numpy()

     print("hot one:",y_train_ohe.shape, y_test_ohe.shape, y_val_ohe.shape)
     return x_train,y_train_ohe, x_val, y_val_ohe

In [17]:

x_train,y_train_ohe, x_val, y_val_ohe = prepa()
BATCH_SIZE = 32
def augmentation(x_train,y_train_ohe, x_val, y_val_ohe ):
    # Create train generator.
    train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip = 'true')
    train_generator = train_datagen.flow(x_train, y_train_ohe, shuffle=False,
                                        batch_size=BATCH_SIZE, seed=1)

    # Create validation generator
    val_datagen = ImageDataGenerator(rescale = 1./255)
    val_generator = train_datagen.flow(x_val, y_val_ohe, shuffle=False,
                                    batch_size=BATCH_SIZE, seed=1)

    return train_generator, val_generator

FileNotFoundError: [Errno 2] No such file or directory: './dog-breed-identification/labels.csv'